# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846769266599                   -0.70    4.8         
  2   -7.852306955383       -2.26       -1.53    1.0   32.0ms
  3   -7.852612566802       -3.51       -2.55    1.5   35.3ms
  4   -7.852645868027       -4.48       -2.86    2.5   44.3ms
  5   -7.852646466512       -6.22       -3.14    1.0   32.3ms
  6   -7.852646678051       -6.67       -3.98    1.0   32.7ms
  7   -7.852646686292       -8.08       -5.06    1.5   37.8ms
  8   -7.852646686720       -9.37       -5.41    2.2   42.2ms
  9   -7.852646686729      -11.05       -6.17    1.0   32.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846906875821                   -0.70           5.0         
  2   -7.852551710341       -2.25       -1.64   0.80    2.0    335ms
  3   -7.852637668383       -4.07       -2.75   0.80    1.0   29.4ms
  4   -7.852646573826       -5.05       -3.29   0.80    2.0   38.1ms
  5   -7.852646678536       -6.98       -4.18   0.80    1.2   31.5ms
  6   -7.852646686333       -8.11       -4.86   0.80    2.0   36.8ms
  7   -7.852646686720       -9.41       -5.71   0.80    1.8   36.4ms
  8   -7.852646686730      -11.02       -6.89   0.80    2.2   38.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.414801e+01     3.038577e+00
 * time: 0.5965800285339355
     1     1.633490e+00     1.771345e+00
 * time: 0.844757080078125
     2    -1.655430e+00     2.109631e+00
 * time: 0.8749830722808838
     3    -3.576458e+00     1.886699e+00
 * time: 0.9181339740753174
     4    -5.159662e+00     1.599442e+00
 * time: 0.9617490768432617
     5    -6.702972e+00     9.218084e-01
 * time: 1.0052158832550049
     6    -6.974767e+00     1.055165e+00
 * time: 1.0357248783111572
     7    -7.505580e+00     8.887503e-01
 * time: 1.0663208961486816
     8    -7.624156e+00     1.011586e+00
 * time: 1.0970709323883057
     9    -7.707179e+00     8.399860e-01
 * time: 1.127655029296875
    10    -7.753750e+00     1.045253e+00
 * time: 1.158276081085205
    11    -7.795560e+00     8.402908e-01
 * time: 1.1889588832855225
    12    -7.822783e+00     2.939465e-01
 * time: 1.2329258918762207
    13    -7.838398e+00     2.450647e-01
 * time: 1.2633559703826

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846799503490                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645896896                   -1.64         
  2   -7.852646686730       -6.10       -3.70    2.07s
  3   -7.852646686730      -13.34       -7.24    186ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.234788284773582e-7
|ρ_newton - ρ_scfv| = 1.4739477373220953e-7
|ρ_newton - ρ_dm|   = 8.522087296667321e-10
